In [2]:
import os, argparse
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pytz import timezone
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.backends.cudnn as cudnn
import torchvision
import wandb

import datasets
# from utils import select_device, natural_keys, gazeto3d, angular, getArch
from utils import select_device, natural_keys, gazeto3d, angular, getArch
from model import L2CS

In [3]:
wandb.init(project="smode_rdata_reverse_sign")

wandb: Currently logged in as: synthesis-ai (use `wandb login --relogin` to force relogin)


In [4]:
# check if we have the correct number of checkpoint files 
ppath ='/project/results/soutput/snapshots/' 
for fold in range(15):
    foldstr = f"fold{fold:0>2}"
    cpath =os.path.join(ppath, foldstr)
    files = os.listdir(cpath)
    print(len(files), end=" ")

61 61 61 61 61 61 61 61 61 61 61 61 61 61 61 

In [5]:
def parse_args():
    """Parse input arguments."""
    parser = argparse.ArgumentParser(
        description='Gaze estimation using L2CSNet .')
     # Gaze360
    parser.add_argument(
        '--gaze360image_dir', dest='gaze360image_dir', help='Directory path for gaze images.',
        default='datasets/Gaze360/Image', type=str)
    parser.add_argument(
        '--gaze360label_dir', dest='gaze360label_dir', help='Directory path for gaze labels.',
        default='datasets/Gaze360/Label/test.label', type=str)
    # mpiigaze
    parser.add_argument(
        '--gazeMpiimage_dir', dest='gazeMpiimage_dir', help='Directory path for gaze images.',
        default='datasets/MPIIFaceGaze/Image', type=str)
    parser.add_argument(
        '--gazeMpiilabel_dir', dest='gazeMpiilabel_dir', help='Directory path for gaze labels.',
        default='datasets/MPIIFaceGaze/Label', type=str)
    # Important args -------------------------------------------------------------------------------------------------------
    # ----------------------------------------------------------------------------------------------------------------------
    parser.add_argument(
        '--dataset', dest='dataset', help='gaze360, mpiigaze',
        default= "gaze360", type=str)
    parser.add_argument(
        '--snapshot', dest='snapshot', help='Path to the folder contains models.', 
        default='output/snapshots/L2CS-gaze360-_loader-180-4-lr', type=str)
    parser.add_argument(
        '--evalpath', dest='evalpath', help='path for the output evaluating gaze test.',
        default="evaluation/L2CS-gaze360-_loader-180-4-lr", type=str)
    parser.add_argument(
        '--gpu',dest='gpu_id', help='GPU device id to use [0]',
        default="0", type=str)
    parser.add_argument(
        '--batch_size', dest='batch_size', help='Batch size.',
        default=100, type=int)
    parser.add_argument(
        '--arch', dest='arch', help='Network architecture, can be: ResNet18, ResNet34, [ResNet50], ''ResNet101, ResNet152, Squeezenet_1_0, Squeezenet_1_1, MobileNetV2',
        default='ResNet50', type=str)
    # ---------------------------------------------------------------------------------------------------------------------
    # Important args ------------------------------------------------------------------------------------------------------
    args = parser.parse_args()
    return args

In [1]:

args = argparse.Namespace()
args.gazeMpiimage_dir = '/project/data/Image'  #real data 
args.gazeMpiilabel_dir = '/project/data/Label'  #real label
args.output = '/project/results/soutput/snapshots/'
args.dataset = 'mpiigaze'
args.snapshot='/project/results/soutput/snapshots/'
args.evalpath = '/project/results/sroutput/evaluation/'
args.gpu_id = '0,1,2,3'
args.gpu_id = '0'
args.batch_size = 20
args.arch = 'ResNet50'
args.bins=28
args.angle = 180
args.bin_width = 4


NameError: name 'argparse' is not defined

In [7]:
batch_size=args.batch_size
arch=args.arch
data_set=args.dataset
evalpath =args.evalpath
snapshot_path = args.snapshot
bins=args.bins
angle=args.angle
bin_width=args.bin_width

In [8]:
# args = parse_args()
cudnn.enabled = True
gpu = select_device(args.gpu_id, batch_size=args.batch_size)
transformations = transforms.Compose([
    transforms.Resize(448),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

model_used=getArch(arch, bins)  #resnet50 and 28 bins

In [9]:
# fold=2
# folder = os.listdir(os.path.join(snapshot_path, "fold" + f'{fold:0>2}'))
# folder.sort(key=natural_keys)
# folder.pop(-1)  #remove the tensorboard file
# # print(folder)
# epochs = folder[3]
# os.path.join(snapshot_path+"fold"+f'{fold:0>2}', epochs)

In [10]:
# print(evalpath, snapshot_path)

In [11]:
# print(data_set)

In [12]:
%%time
all_MAE = []
# tosave={}
for fold in range(15):
    print(f"fold={fold}")
    epoch_values=[]
    mae_values=[]


    now = datetime.utcnow()
    now = now.astimezone(timezone('US/Pacific'))
    date_format='%m/%d/%Y %H:%M:%S'
    now = now.strftime(date_format)
    
    print(args.gazeMpiilabel_dir)
    folder = os.listdir(args.gazeMpiilabel_dir)
    folder.sort()  #individual label files
    testlabelpathcombined = [os.path.join(args.gazeMpiilabel_dir, j) for j in folder] 
    
    gaze_dataset=datasets.Mpiigaze(testlabelpathcombined, args.gazeMpiimage_dir, transformations, False, angle, fold)

    test_loader = torch.utils.data.DataLoader(
        dataset=gaze_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True)

    fold_path = os.path.join(evalpath, 'fold' + f'{fold:0>2}'+'/')  #for evaluation
#     print(f"fold_path is {fold_path}")
    if not os.path.exists(fold_path):
        os.makedirs(fold_path)

#     if not os.path.exists(os.path.join(evalpath, f"fold"+str(fold))):
#         os.makedirs(os.path.join(evalpath, f"fold"+str(fold)))

    # list all epoch for testing
    folder = os.listdir(os.path.join(snapshot_path, "fold" + f'{fold:0>2}'))
    folder.sort(key=natural_keys)
    folder.pop(-1)  #remove the tensorboard file, now all snapshot files
#     print(f"folder={folder}")
                    

    softmax = nn.Softmax(dim=1)
#     with open(os.path.join(evalpath, os.path.join("fold"+f'{fold:0>2}', data_set+".log")), 'w') as outfile:
        
    configuration = (f"\ntest configuration equal gpu_id={gpu}, batch_size={batch_size}, model_arch={arch}\n"
                     f"Start testing dataset={data_set}, FOLD={fold} --{now}---------")
    print(configuration)
#     tosave['time':now]
#     outfile.write(configuration)
    epoch_list=[]
    avg_MAE=[]
    for epoch in folder: 
        print(f"entering epoch={epoch}")
        model=model_used
        checkpoint = torch.load(os.path.join(snapshot_path+"fold"+f'{fold:0>2}', epoch))
        saved_state_dict = checkpoint['model_state_dict']
        model= nn.DataParallel(model,device_ids=[0])
        model.load_state_dict(saved_state_dict)
        model.cuda(gpu)
        model.eval()
        total = 0
        idx_tensor = [idx for idx in range(28)]
        idx_tensor = torch.FloatTensor(idx_tensor).cuda(gpu)
        avg_error = .0
        with torch.no_grad():
            for j, (images, labels, cont_labels, name) in enumerate(test_loader):
                images = Variable(images).cuda(gpu)
                total += cont_labels.size(0)

                label_pitch = cont_labels[:,0].float()*np.pi/180
                label_yaw = cont_labels[:,1].float()*np.pi/180

                gaze_pitch, gaze_yaw = model(images)

                # Binned predictions
                _, pitch_bpred = torch.max(gaze_pitch.data, 1)
                _, yaw_bpred = torch.max(gaze_yaw.data, 1)

                # Continuous predictions
                pitch_predicted = softmax(gaze_pitch)
                yaw_predicted = softmax(gaze_yaw)

                # mapping from binned (0 to 28) to angels (-42 to 42)                
                pitch_predicted = \
                    torch.sum(pitch_predicted * idx_tensor, 1).cpu() * 3 - 42
                yaw_predicted = \
                    torch.sum(yaw_predicted * idx_tensor, 1).cpu() * 3 - 42

                pitch_predicted = pitch_predicted*np.pi/180
                yaw_predicted = yaw_predicted*np.pi/180

                for p,y,pl,yl in zip(pitch_predicted, yaw_predicted, label_pitch, label_yaw):
                    pl, yl = yl, pl
#                     yl = yl*(-1.0)
                    avg_error += angular(gazeto3d([p,y]), gazeto3d([pl,yl]))

        x = ''.join(filter(lambda i: i.isdigit(), epoch))
#             print(f"x={x}")
        epoch_list.append(x)
        mean_mae = avg_error/total  #mean mae over the 3000 iamges
        avg_MAE.append(mean_mae)  
#             print(f"total={total}")
        
        loger = f"[{epoch}---{args.dataset}] Total Num:{total},MAE:{mean_mae}"
#         outfile.write(loger)
        print(loger)
#             print(f"done epoch={epochs}")
        epochn = int(x)
#         wandb.log({'epoch':epochn,'total':total, 'MAE':mean_mae, 'FOLD':fold })
#         tosave ={'epoch':epochn,'total':total, 'MAE':mean_mae, 'FOLD':fold }
        epoch_values.append(epochn)
        mae_values.append(mean_mae)
# #         wandb.log(tosave)
    all_MAE.append(avg_MAE)
    data = [[x, y] for (x, y) in zip(epoch_values, mae_values)]
    table = wandb.Table(data=data, columns = ["x", "y"])
    wandb.log({"epoch_losses" : wandb.plot.line(table, "x", "y",
               title="Epoch loss for each fold")})
    
#     fig = plt.figure(figsize=(14, 8))        
#     plt.xlabel('epoch')
#     plt.ylabel('avg')
#     plt.title('Gaze angular error')
#     plt.legend()
#     plt.plot(epoch_list, avg_MAE, color='k', label='mae')
#     fig.savefig(os.path.join(evalpath, os.path.join("fold"+f'{fold:0>2}', data_set+".png")), format='png')
#     # plt.show() 

fold=0
/project/data/Label
0 items removed from dataset that have an angle > 180

test configuration equal gpu_id=cuda:0, batch_size=20, model_arch=ResNet50
Start testing dataset=mpiigaze, FOLD=0 --06/17/2022 17:09:19---------
entering epoch=epoch_1.pkl
[epoch_1.pkl---mpiigaze] Total Num:3000,MAE:13.965764770150047
entering epoch=epoch_2.pkl
[epoch_2.pkl---mpiigaze] Total Num:3000,MAE:13.310476886626795
entering epoch=epoch_3.pkl
[epoch_3.pkl---mpiigaze] Total Num:3000,MAE:13.964950913254011
entering epoch=epoch_4.pkl
[epoch_4.pkl---mpiigaze] Total Num:3000,MAE:13.938010759117999
entering epoch=epoch_5.pkl
[epoch_5.pkl---mpiigaze] Total Num:3000,MAE:13.047713832480783
entering epoch=epoch_6.pkl
[epoch_6.pkl---mpiigaze] Total Num:3000,MAE:12.701151995302467
entering epoch=epoch_7.pkl
[epoch_7.pkl---mpiigaze] Total Num:3000,MAE:12.697705981177707
entering epoch=epoch_8.pkl
[epoch_8.pkl---mpiigaze] Total Num:3000,MAE:13.714712629553384
entering epoch=epoch_9.pkl
[epoch_9.pkl---mpiigaze] T

[epoch_40.pkl---mpiigaze] Total Num:3000,MAE:12.782668427506879
entering epoch=epoch_41.pkl
[epoch_41.pkl---mpiigaze] Total Num:3000,MAE:12.528315243240835
entering epoch=epoch_42.pkl
[epoch_42.pkl---mpiigaze] Total Num:3000,MAE:13.528603275960936
entering epoch=epoch_43.pkl
[epoch_43.pkl---mpiigaze] Total Num:3000,MAE:13.346865133012665
entering epoch=epoch_44.pkl
[epoch_44.pkl---mpiigaze] Total Num:3000,MAE:13.413585831852672
entering epoch=epoch_45.pkl
[epoch_45.pkl---mpiigaze] Total Num:3000,MAE:13.614334813142497
entering epoch=epoch_46.pkl
[epoch_46.pkl---mpiigaze] Total Num:3000,MAE:13.074439279211374
entering epoch=epoch_47.pkl
[epoch_47.pkl---mpiigaze] Total Num:3000,MAE:14.369535375733742
entering epoch=epoch_48.pkl
[epoch_48.pkl---mpiigaze] Total Num:3000,MAE:13.312857561623884
entering epoch=epoch_49.pkl
[epoch_49.pkl---mpiigaze] Total Num:3000,MAE:13.248739523604602
entering epoch=epoch_50.pkl
[epoch_50.pkl---mpiigaze] Total Num:3000,MAE:13.576089233532421
entering epoch=e

[epoch_11.pkl---mpiigaze] Total Num:3000,MAE:16.054214682406755
entering epoch=epoch_12.pkl
[epoch_12.pkl---mpiigaze] Total Num:3000,MAE:16.180795668165977
entering epoch=epoch_13.pkl
[epoch_13.pkl---mpiigaze] Total Num:3000,MAE:17.19262322626391
entering epoch=epoch_14.pkl
[epoch_14.pkl---mpiigaze] Total Num:3000,MAE:16.439022927689344
entering epoch=epoch_15.pkl
[epoch_15.pkl---mpiigaze] Total Num:3000,MAE:15.710511153859722
entering epoch=epoch_16.pkl
[epoch_16.pkl---mpiigaze] Total Num:3000,MAE:16.642114310352476
entering epoch=epoch_17.pkl
[epoch_17.pkl---mpiigaze] Total Num:3000,MAE:16.674246731340723
entering epoch=epoch_18.pkl
[epoch_18.pkl---mpiigaze] Total Num:3000,MAE:16.195755723317045
entering epoch=epoch_19.pkl
[epoch_19.pkl---mpiigaze] Total Num:3000,MAE:15.20088765381742
entering epoch=epoch_20.pkl
[epoch_20.pkl---mpiigaze] Total Num:3000,MAE:16.14788538771386
entering epoch=epoch_21.pkl
[epoch_21.pkl---mpiigaze] Total Num:3000,MAE:15.679065371670417
entering epoch=epoc

[epoch_46.pkl---mpiigaze] Total Num:3000,MAE:11.295800844808488
entering epoch=epoch_47.pkl
[epoch_47.pkl---mpiigaze] Total Num:3000,MAE:11.618644901698085
entering epoch=epoch_48.pkl
[epoch_48.pkl---mpiigaze] Total Num:3000,MAE:11.271511730062016
entering epoch=epoch_49.pkl
[epoch_49.pkl---mpiigaze] Total Num:3000,MAE:11.790277212104202
entering epoch=epoch_50.pkl
[epoch_50.pkl---mpiigaze] Total Num:3000,MAE:11.408949035936743
entering epoch=epoch_51.pkl
[epoch_51.pkl---mpiigaze] Total Num:3000,MAE:11.397646834649354
entering epoch=epoch_52.pkl
[epoch_52.pkl---mpiigaze] Total Num:3000,MAE:11.776231918954913
entering epoch=epoch_53.pkl
[epoch_53.pkl---mpiigaze] Total Num:3000,MAE:11.364265948897819
entering epoch=epoch_54.pkl
[epoch_54.pkl---mpiigaze] Total Num:3000,MAE:12.02860218738854
entering epoch=epoch_55.pkl
[epoch_55.pkl---mpiigaze] Total Num:3000,MAE:11.51496678960123
entering epoch=epoch_56.pkl
[epoch_56.pkl---mpiigaze] Total Num:3000,MAE:12.2696154609726
entering epoch=epoch

[epoch_19.pkl---mpiigaze] Total Num:3000,MAE:19.306575908579603
entering epoch=epoch_20.pkl
[epoch_20.pkl---mpiigaze] Total Num:3000,MAE:18.91801575131086
entering epoch=epoch_21.pkl
[epoch_21.pkl---mpiigaze] Total Num:3000,MAE:19.043578459361143
entering epoch=epoch_22.pkl
[epoch_22.pkl---mpiigaze] Total Num:3000,MAE:18.234151902200434
entering epoch=epoch_23.pkl
[epoch_23.pkl---mpiigaze] Total Num:3000,MAE:18.80468143817583
entering epoch=epoch_24.pkl
[epoch_24.pkl---mpiigaze] Total Num:3000,MAE:18.83699370631027
entering epoch=epoch_25.pkl
[epoch_25.pkl---mpiigaze] Total Num:3000,MAE:18.492965213930898
entering epoch=epoch_26.pkl
[epoch_26.pkl---mpiigaze] Total Num:3000,MAE:18.927706435863954
entering epoch=epoch_27.pkl
[epoch_27.pkl---mpiigaze] Total Num:3000,MAE:19.610083029926976
entering epoch=epoch_28.pkl
[epoch_28.pkl---mpiigaze] Total Num:3000,MAE:18.483583468253677
entering epoch=epoch_29.pkl
[epoch_29.pkl---mpiigaze] Total Num:3000,MAE:19.12954385044353
entering epoch=epoch

[epoch_53.pkl---mpiigaze] Total Num:3000,MAE:15.693701150906005
entering epoch=epoch_54.pkl
[epoch_54.pkl---mpiigaze] Total Num:3000,MAE:16.241718796199486
entering epoch=epoch_55.pkl
[epoch_55.pkl---mpiigaze] Total Num:3000,MAE:16.150276022753093
entering epoch=epoch_56.pkl
[epoch_56.pkl---mpiigaze] Total Num:3000,MAE:16.251759000888754
entering epoch=epoch_57.pkl
[epoch_57.pkl---mpiigaze] Total Num:3000,MAE:16.4159729503721
entering epoch=epoch_58.pkl
[epoch_58.pkl---mpiigaze] Total Num:3000,MAE:15.364603578490579
entering epoch=epoch_59.pkl
[epoch_59.pkl---mpiigaze] Total Num:3000,MAE:16.080301233860386
entering epoch=epoch_60.pkl
[epoch_60.pkl---mpiigaze] Total Num:3000,MAE:16.18022121400477
fold=8
/project/data/Label
0 items removed from dataset that have an angle > 180

test configuration equal gpu_id=cuda:0, batch_size=20, model_arch=ResNet50
Start testing dataset=mpiigaze, FOLD=8 --06/17/2022 19:05:00---------
entering epoch=epoch_1.pkl
[epoch_1.pkl---mpiigaze] Total Num:3000,M

[epoch_28.pkl---mpiigaze] Total Num:3000,MAE:12.245230796986844
entering epoch=epoch_29.pkl
[epoch_29.pkl---mpiigaze] Total Num:3000,MAE:12.002905976223369
entering epoch=epoch_30.pkl
[epoch_30.pkl---mpiigaze] Total Num:3000,MAE:12.146610933781439
entering epoch=epoch_31.pkl
[epoch_31.pkl---mpiigaze] Total Num:3000,MAE:12.437113196995469
entering epoch=epoch_32.pkl
[epoch_32.pkl---mpiigaze] Total Num:3000,MAE:13.180926994011541
entering epoch=epoch_33.pkl
[epoch_33.pkl---mpiigaze] Total Num:3000,MAE:12.795815531769493
entering epoch=epoch_34.pkl
[epoch_34.pkl---mpiigaze] Total Num:3000,MAE:11.94164748632335
entering epoch=epoch_35.pkl
[epoch_35.pkl---mpiigaze] Total Num:3000,MAE:12.634665807483431
entering epoch=epoch_36.pkl
[epoch_36.pkl---mpiigaze] Total Num:3000,MAE:12.782748744975674
entering epoch=epoch_37.pkl
[epoch_37.pkl---mpiigaze] Total Num:3000,MAE:12.45596593809094
entering epoch=epoch_38.pkl
[epoch_38.pkl---mpiigaze] Total Num:3000,MAE:12.724993518425125
entering epoch=epo

[epoch_8.pkl---mpiigaze] Total Num:3000,MAE:11.995204733723554
entering epoch=epoch_9.pkl
[epoch_9.pkl---mpiigaze] Total Num:3000,MAE:12.348793492251689
entering epoch=epoch_10.pkl
[epoch_10.pkl---mpiigaze] Total Num:3000,MAE:12.292687059828838
entering epoch=epoch_11.pkl
[epoch_11.pkl---mpiigaze] Total Num:3000,MAE:13.059751485221254
entering epoch=epoch_12.pkl
[epoch_12.pkl---mpiigaze] Total Num:3000,MAE:12.72391756081714
entering epoch=epoch_13.pkl
[epoch_13.pkl---mpiigaze] Total Num:3000,MAE:13.06912789251609
entering epoch=epoch_14.pkl
[epoch_14.pkl---mpiigaze] Total Num:3000,MAE:12.069215167718196
entering epoch=epoch_15.pkl
[epoch_15.pkl---mpiigaze] Total Num:3000,MAE:12.666568554732024
entering epoch=epoch_16.pkl
[epoch_16.pkl---mpiigaze] Total Num:3000,MAE:12.716631392521238
entering epoch=epoch_17.pkl
[epoch_17.pkl---mpiigaze] Total Num:3000,MAE:13.135636405154381
entering epoch=epoch_18.pkl
[epoch_18.pkl---mpiigaze] Total Num:3000,MAE:12.610325406064378
entering epoch=epoch_

[epoch_42.pkl---mpiigaze] Total Num:3000,MAE:17.183799454131815
entering epoch=epoch_43.pkl
[epoch_43.pkl---mpiigaze] Total Num:3000,MAE:16.80887357199948
entering epoch=epoch_44.pkl
[epoch_44.pkl---mpiigaze] Total Num:3000,MAE:17.00301173394433
entering epoch=epoch_45.pkl
[epoch_45.pkl---mpiigaze] Total Num:3000,MAE:16.154422575804738
entering epoch=epoch_46.pkl
[epoch_46.pkl---mpiigaze] Total Num:3000,MAE:17.573624236771987
entering epoch=epoch_47.pkl
[epoch_47.pkl---mpiigaze] Total Num:3000,MAE:16.166261824789537
entering epoch=epoch_48.pkl
[epoch_48.pkl---mpiigaze] Total Num:3000,MAE:16.5159660857
entering epoch=epoch_49.pkl
[epoch_49.pkl---mpiigaze] Total Num:3000,MAE:17.375582322104034
entering epoch=epoch_50.pkl
[epoch_50.pkl---mpiigaze] Total Num:3000,MAE:17.453548690274825
entering epoch=epoch_51.pkl
[epoch_51.pkl---mpiigaze] Total Num:3000,MAE:16.425911157173328
entering epoch=epoch_52.pkl
[epoch_52.pkl---mpiigaze] Total Num:3000,MAE:17.47573484550906
entering epoch=epoch_53.

[epoch_7.pkl---mpiigaze] Total Num:3000,MAE:15.359396749798586
entering epoch=epoch_8.pkl
[epoch_8.pkl---mpiigaze] Total Num:3000,MAE:16.744807709166697
entering epoch=epoch_9.pkl
[epoch_9.pkl---mpiigaze] Total Num:3000,MAE:16.39557869569305
entering epoch=epoch_10.pkl
[epoch_10.pkl---mpiigaze] Total Num:3000,MAE:16.94231357521441
entering epoch=epoch_11.pkl
[epoch_11.pkl---mpiigaze] Total Num:3000,MAE:16.314846991964664
entering epoch=epoch_12.pkl
[epoch_12.pkl---mpiigaze] Total Num:3000,MAE:16.73608532891329
entering epoch=epoch_13.pkl
[epoch_13.pkl---mpiigaze] Total Num:3000,MAE:17.13277376029165
entering epoch=epoch_14.pkl
[epoch_14.pkl---mpiigaze] Total Num:3000,MAE:16.544510591484833
entering epoch=epoch_15.pkl
[epoch_15.pkl---mpiigaze] Total Num:3000,MAE:16.787302277241984
entering epoch=epoch_16.pkl
[epoch_16.pkl---mpiigaze] Total Num:3000,MAE:16.883606724138374
entering epoch=epoch_17.pkl
[epoch_17.pkl---mpiigaze] Total Num:3000,MAE:16.55770272441199
entering epoch=epoch_18.pk

In [13]:
all_MAE=np.array(all_MAE)

In [14]:

print(all_MAE.shape)
print(all_MAE.mean(axis=0))
print(all_MAE.mean(axis=1))
print(all_MAE.mean(axis=0).argmin()+1 ,all_MAE.mean(axis=0).min())

(15, 60)
[12.73320015 12.66976048 13.29805501 13.49525361 13.49816027 13.51472531
 13.30842204 14.09889858 14.05949045 13.85637639 13.90967756 14.32806288
 14.10930814 14.10788745 14.14207063 14.32835782 14.23040513 14.34655454
 14.26615159 14.00458119 14.25726836 14.20687374 14.35447629 14.36525256
 14.18271497 14.39796341 14.50344808 14.32886481 14.53107158 14.59748938
 14.40030267 14.69282299 14.43906777 14.34944433 14.60067985 14.61058843
 14.41827577 14.57232955 14.66060559 14.62001057 14.61284852 14.69078068
 14.71025778 14.62712429 14.61639218 14.69013644 14.78735022 14.63679245
 14.85162752 14.73501859 14.60493425 14.88924773 14.64028477 14.7535466
 14.90905922 14.91213516 14.91818184 14.808922   14.90649021 14.76218099]
[13.36636655 12.58951346 14.97902285 16.07620969 10.7099251  13.46431787
 18.8250614  15.43233654 13.56937345 12.01918444 16.02104838 12.47781844
 16.52140525 12.06828396 16.99469796]
2 12.669760482027039
